# Exercises: Recurrent Neural Networks

## Read the data

In [ ]:
nov_path = 'kaijin_nijumenso.txt'

with open(nov_path, 'r') as f:
    nov_text = f.read()

## Preprocessiong

In [ ]:
import re

text = re.sub("《[^》]+》", "", nov_text)  # remove <brackets>
text = re.sub("［[^］]+］", "", text) # remove [brackets]
text = re.sub("[｜ 　]", "", text) # remove | and spaces

## Setting up the hyperparameters

In [ ]:
n_rnn = 20 # number of characters to consider
batch_size = 256 # number of sequences to consider at once
n_epochs = 100 # number of epochs to train for
n_hidden = 256 # number of hidden units in the RNN

## word2vec

In [1]:
import numpy as np

# create a index to character mapping
chars = list(set(text))
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}

# cut the text in semi-redundant sequences of n_rnn characters
time_chars = []
next_chars = []
for i in range(0, len(text) - n_rnn):
    time_chars.append(text[i: i + n_rnn])
    next_chars.append(text[i + n_rnn])

# create a vectorized representation of the sequences
X = np.zeros((len(time_chars), n_rnn, len(chars)), dtype=np.bool)
y = np.zeros((len(time_chars), len(chars)), dtype=np.bool)
for i, time_char in enumerate(time_chars):
    for t, char in enumerate(time_char):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

NameError: name 'text' is not defined

## Creating the model

from keras.models import Sequential
from keras.layers import Dense, GRU

model = Sequential()
model.add(GRU(n_hidden, input_shape=(n_rnn, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

## Defining the setence generator function

In [ ]:
from keras.callbacks import LambdaCallback
def on_epoch_end(epochs, logs):
    if(epochs == n_epochs-1 or epochs == 0):
        print('----- Generating text after Epoch: %d' % epochs)
        beta = 5 # temperature
        prev_text = text[0: n_rnn] # seed text
        created_text = prev_text

        print('----- Generating with seed: "' + prev_text + '"')
        for i in range(400):
            # one-hot encode the characters
            X_pred = np.zeros((1, n_rnn, len(chars)))
            for t, char in enumerate(prev_text):
                X_pred[0, t, char_indices[char]] = 1.

            # predict the next character
            preds = model.predict(X_pred, verbose=0)[0]
            p_power = np.asarray(preds).astype('float64')
            p_power = p_power ** beta # modify the distribution
            next_index = np.random.choice(len(p_power), p=p_power/np.sum(p_power))
            next_char = indices_char[next_index]

            created_text += next_char
            prev_text = prev_text[1:] + next_char 
        print(created_text)
        print()

# set up the callbacks
epoch_end_callback = LambdaCallback(on_epoch_end=on_epoch_end)

## Training the model

In [ ]:
history = model.fit(X, y, batch_size=batch_size, epochs=n_epochs, callbacks=[epoch_end_callback])

## History

In [ ]:
loss = history.history['loss']

import matplotlib.pyplot as plt
plt.plot(loss, label='loss')
plt.legend()
plt.show()